In [5]:
import os
import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import tensorflow as tf
from tensorflow import keras
import transformers
from transformers import TextClassificationPipeline, AutoTokenizer, TFAutoModelForSequenceClassification

## Enter your text

Config params:

In [6]:
# resources folder
resources_path = "../res"
datasets_path = os.path.join(resources_path, "datasets")
models_path = os.path.join(resources_path, "models")

In [8]:
num_classes = 24

model = TFAutoModelForSequenceClassification.from_pretrained(os.path.join(models_path, "bert_finetuned"))

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, top_k = num_classes)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at ../res\models\bert_finetuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.
c:\Users\Jorge\anaconda3\envs\translations_hp\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Jorge\.cache\huggingface\hub\models--bert-base-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To s

Enter text for first-aid recommendations:

In [9]:
input = "My kid has fallen overboard from my boat. We rescued him from the water but now he is unconscious, what should I do?"

In [10]:
def predict_disease(text : str) -> str:
    return pipe(text)[0][:2]

In [11]:
conditions = predict_disease(input)
conditions

[{'label': 'drowning', 'score': 0.9311556220054626},
 {'label': 'fracture', 'score': 0.008220311254262924}]

In [12]:
conditions = conditions[0]['label']

Functions to search condition

In [13]:
def load_data(filename):
    # Load JSON data from a file
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

def get_advice_by_tag(data, tag):
    # Search for the tag in the data and return the corresponding advice
    for entry in data['intents']:
        if entry['tag'].lower() == tag.lower():
            return entry['responses']
    return "No advice found for this tag."

In [15]:
filename = os.path.join(datasets_path, 'Medical_Aid.json')

data = load_data(filename)

advice = get_advice_by_tag(data, conditions)
print("Advice for", conditions, ":\n", advice[0])

Advice for drowning :
 1)Place your ear next to the person's mouth and nose. Do you feel air on your cheek? 2)Look to see if the person's chest is moving.If the Person is Not Breathing, Check Pulse. 3)Check the person's pulse for 10 seconds.If There is No Pulse, Start CPR.


In [17]:
from transformers import MarianMTModel, MarianTokenizer

# Load pre-trained model and tokenizer
model_name = "Helsinki-NLP/opus-mt-en-es"
model = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(model_name)

c:\Users\Jorge\anaconda3\envs\translations_hp\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [18]:
# Tokenize the input text
input_ids = tokenizer.encode(advice[0], return_tensors="pt")

In [19]:
# Perform translation
translated_tokens = model.generate(input_ids)

In [21]:
# Decode the translated tokens
translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
print("Translated text:", translated_text)

Translated text: 1) Coloque su oído junto a la boca y la nariz de la persona. ¿Siente aire en la mejilla? 2) Busque si el pecho de la persona se está moviendo.Si la persona no respira, compruebe el pulso. 3) Compruebe el pulso de la persona durante 10 segundos.Si no hay pulso, inicie la RCP.
